### Portfolio Optimization

Portfolio optimization is a particular application of optimization aimed at building portfolios of stocks and other assets that are "optimal" (and perhaps also robust) in some sense. It is an incredibly deep and complex subject that will will not attempt to cover here.

Here we consider a simple model to determine how to invest a budget $B$ across a set of $n$ assets or stocks in order to maximize profit and minimize risk. Here, $\mu \in \mathbb{R}^n$ is the vector of average returns of the $n$ stocks. $x \in \mathbb{R}^n$ is the vector representing how much we have decided to allocate across the $n$ assets. In this particular model, we will tradeoff the **expected return** $\mu^Tx$ against the **risk** $-x^T\Sigma x$ in a portfolio of $n$ stocks/assets. Here, we use an ideosyncratic risk model where

\begin{equation}
\Sigma = FF^T + D^2
\end{equation}

Here $FF^T$ is a low rank matrix and $D^2$ is a diagonal matrix with all positive entries.

Our constraints are that $\sum_{i=1}^n x_i = B$ for some budget $B$, and that $x \geq 0$ (aka our portfolio is constrained to be **long-only**).

The final problem is:

\begin{equation*}
  \begin{aligned}
    &\text{maximize} && \mu^Tx - \gamma x^T(FF^T + D^2)x\\
    &\text{subject to} && \textbf{1}^Tx = B \\
    &                  && x \geq 0
  \end{aligned}
\end{equation*}

with variable $x$, tradeoff parameter $\gamma$, and constants $\mu, F, D$, and $B$.


In [ ]:
import cvxpy as cp
import numpy as np
import scipy as sp

# setup

problemID = "portfolio_0"
prob = None
opt_val = None

# Variable declarations

import scipy.sparse as sps

np.random.seed(0)
m = 500
n = 50000
density = 0.1

mu = np.exp(0.01*np.random.randn(n))-1  # returns
D = np.random.rand(n)/10;               # idiosyncratic risk
F = sps.rand(n,m,density)                # factor model
F.data = np.random.randn(len(F.data))/10
gamma = 1
B = 1

x = cp.Variable(n)


# Problem construction

f = mu.T*x - gamma*(cp.sum_squares(F.T.dot(x)) +
                    cp.sum_squares(cp.mul_elemwise(D, x)))
# f = mu.T*x - gamma*cp.quad_form(x, sigma)
C = [cp.sum_entries(x) == B,
     x >= 0]

prob = cp.Problem(cp.Maximize(f), C)


# Problem collection

# Single problem collection
problemDict = {
    "problemID" : problemID,
    "problem"   : prob,
    "opt_val"   : opt_val
}
problems = [problemDict]



# For debugging individual problems:
if __name__ == "__main__":
    def printResults(problemID = "", problem = None, opt_val = None):
        print(problemID)
        problem.solve()
        print("\tstatus: {}".format(problem.status))
        print("\toptimal value: {}".format(problem.value))
        print("\ttrue optimal value: {}".format(opt_val))
    printResults(**problems[0])
